In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1608242786205_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType


 
This notebook will read NY Taxi data , transform it and save it to parquet format 

In [2]:

lazy val schema =
StructType(Array(
StructField("VendorId", IntegerType),
StructField("tpep_pickup_datetime", TimestampType),
StructField("tpep_dropoff_datetime", TimestampType),
StructField("passenger_count", IntegerType),
StructField("trip_distance", FloatType),
StructField("pickup_longitude", FloatType),
StructField("pickup_latitude", FloatType),
StructField("RatecodeID", IntegerType),
StructField("store_and_fwd_flag", StringType),
StructField("dropoff_longitude", FloatType),
StructField("dropoff_latitude", FloatType),
StructField("payment_type", IntegerType),
StructField("fare_amount", FloatType),
StructField("extra", FloatType),
StructField("mta_tax", FloatType),
StructField("tip_amount", FloatType),
StructField("tolls_amount", FloatType),
StructField("improvement_surcharge", FloatType),
StructField("total_amount", FloatType) 
  ))

In [3]:
// Either copy the data to your bucket or read it from the pulic bucket
// aws: after ssh into master: 
// wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-02.csv
// aws s3 cp yellow_tripdata_2016-02.csv s3://mybucket/data/

//var df = spark.read.option("inferSchema", "false").option("header", true).schema(schema).csv("s3://bucketcjm/data/yellow_tripdata_2016-02.csv") 

var df = spark.read.option("inferSchema", "false").option("header", true).schema(schema).csv("s3://nyc-tlc/trip data/yellow_tripdata_2018-01.csv")

In [4]:
df.show

In [5]:
def dropUseless(dataFrame: DataFrame): DataFrame = {
    dataFrame.drop(
      "payment_type",
      "extra",
      "mta_tax",
      "tip_amount",
      "tolls_amount",
      "improvement_surcharge",
      "total_amount")
  }

In [6]:
df = dropUseless(df)

In [7]:
df = df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
df = df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
df = df.withColumnRenamed("VendorID", "vendor_id")
df = df.withColumnRenamed("store_and_fwd_flag", "store_and_fwd")
df = df.withColumnRenamed("RatecodeID", "rate_code")

In [8]:
z.show (df)

In [9]:
val start = unix_timestamp(col("pickup_datetime")).cast(LongType)
val end = unix_timestamp(col("dropoff_datetime")).cast(LongType)
df = df.withColumn("trip_time", (end-start))

In [10]:
// get day of week and hour
val datetime = col("pickup_datetime")
df = df.withColumn("year", year(datetime)).withColumn("month", month(datetime))
      .withColumn("day", dayofmonth(datetime))
      .withColumn("day_of_week", dayofweek(datetime))
      .withColumn("is_weekend",col("day_of_week").isin(1, 7).cast(IntegerType))  // 1: Sunday, 7: Saturday
      .withColumn("hour", hour(datetime))
      .drop("dropoff_datetime","pickup_datetime")

In [11]:
df.schema


In [12]:
// filter out anomalous values
df = df.filter(!($"trip_distance" < 1 and $"fare_amount" > 15 ))
df = df.filter(!($"trip_distance" < 10 and $"fare_amount" > 40 ))
df = df.filter($"trip_distance" > 0 and $"trip_distance" < 100)
df = df.filter($"fare_amount" > 0 and $"fare_amount" < 100)
df = df.filter($"passenger_count" > 0 and $"passenger_count" < 6)
df = df.filter($"pickup_longitude" > -75 and $"pickup_longitude" < -73)
df = df.filter($"dropoff_longitude" > -75 and $"dropoff_longitude" < -73)
df = df.filter($"pickup_latitude" > 40 and $"pickup_latitude" < 42)
df = df.filter($"dropoff_latitude" > 40 and $"dropoff_latitude" < 42)

In [13]:
df.show

In [14]:
// filter out anomouslous trip_time values
df = df.filter(!($"trip_time" < 1000 and $"fare_amount" > 40 ))
df = df.filter($"trip_time" > 10 )
df = df.filter($"trip_time" <  40000)

In [15]:
df.show

In [16]:
// calculate haversine distance  
def addHDistance(dataFrame: DataFrame): DataFrame = {
    val P = math.Pi / 180
    val lat1 = col("pickup_latitude")
    val lon1 = col("pickup_longitude")
    val lat2 = col("dropoff_latitude")
    val lon2 = col("dropoff_longitude")
    val internalValue = (lit(0.5)
      - cos((lat2 - lat1) * P) / 2
      + cos(lat1 * P) * cos(lat2 * P) * (lit(1) - cos((lon2 - lon1) * P)) / 2)
    val hDistance = lit(12734) * asin(sqrt(internalValue))
    dataFrame.withColumn("h_distance", hDistance)
  }

In [17]:
df = addHDistance(df)

In [18]:
df = df.filter(!($"h_distance" < 1 and $"fare_amount" >15))
df = df.filter(!($"h_distance" < 10 and $"fare_amount" > 40 ))
df = df.filter($"h_distance" > 1 and $"h_distance" < 100)

In [19]:
// round off location values
df = df.withColumn("pickup_longitude", round( df("pickup_longitude"),3)).withColumn("pickup_latitude", round( df("pickup_latitude"),3))
df = df.withColumn("dropoff_longitude", round( df("dropoff_longitude"),3)).withColumn("dropoff_latitude", round( df("dropoff_latitude"),3))

In [20]:
df = df.withColumn("h_distance", round( $"h_distance",3))

In [21]:
df.show

In [22]:
df.select("h_distance","fare_amount","trip_time").describe().show

In [23]:
df.schema


In [24]:
df.createOrReplaceTempView("taxi")


In [25]:
%%sql
select hour, avg(fare_amount), avg(h_distance)
from taxi
group by hour order by hour

In [26]:
df.write.mode("overwrite").parquet("s3://bucketcjm/data/pyellow_tripdata_2016-02")


In [27]:
df.unpersist

In [2]:
var tdf = spark.read.parquet("s3://bucketcjm/data/pyellow_tripdata_2016-02") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tdf: org.apache.spark.sql.DataFrame = [vendor_id: int, passenger_count: int ... 16 more fields]


In [3]:
tdf.show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+-------------+----------------+---------------+---------+-------------+-----------------+----------------+-----------+---------+----+-----+---+-----------+----------+----+----------+
|vendor_id|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code|store_and_fwd|dropoff_longitude|dropoff_latitude|fare_amount|trip_time|year|month|day|day_of_week|is_weekend|hour|h_distance|
+---------+---------------+-------------+----------------+---------------+---------+-------------+-----------------+----------------+-----------+---------+----+-----+---+-----------+----------+----+----------+
|        1|              2|          5.9|         -73.788|         40.642|        1|            N|          -73.826|          40.695|       19.5|     1154|2016|    2|  4|          5|         0|  17|     6.784|
|        1|              1|          1.5|         -74.001|         40.717|        1|            N|          -74.017|          40.711|        9.5|      725|2016|

In [4]:
tdf.cache
tdf.createOrReplaceTempView("taxi")
spark.catalog.cacheTable("taxi")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [vendor_id: int, passenger_count: int ... 16 more fields]


In [5]:
tdf.select("h_distance","fare_amount","trip_time").describe().show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+-----------------+
|summary|        h_distance|       fare_amount|        trip_time|
+-------+------------------+------------------+-----------------+
|  count|           8839408|           8839408|          8839408|
|   mean|3.8643447165240135|13.623234854638849|889.0052525010725|
| stddev| 3.906505092446503| 9.992479011059864|631.0750517597542|
|    min|               1.0|              0.01|               11|
|    max|            91.005|             99.99|            39972|
+-------+------------------+------------------+-----------------+



In [6]:
tdf.groupBy("hour").avg("fare_amount", "h_distance").orderBy("hour").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+------------------+
|hour|  avg(fare_amount)|   avg(h_distance)|
+----+------------------+------------------+
|   0|14.179643317334271| 4.458087735046495|
|   1|13.827095010892949| 4.429371479029097|
|   2|13.713627421494186|  4.43074106685054|
|   3|14.264518245221398| 4.697226917381569|
|   4|16.266169431476094| 5.517976423449112|
|   5|17.538526357708626|6.1107537361347966|
|   6|14.102223913972002| 4.668029960670734|
|   7|12.815018112487849|3.7610465452425803|
|   8|13.004404053988457| 3.429177667894983|
|   9|13.080668873012876| 3.355509792620909|
|  10|13.245307067161042|3.4511313568688626|
|  11|13.313469174145773| 3.441582293868121|
|  12|13.436498532200424|3.4871494610994267|
|  13| 13.83863286030106| 3.675348011628207|
|  14|14.253723045043053| 3.825762018452578|
|  15|14.206508132846748|3.8149681636721944|
|  16|14.137146803493694| 3.895133809979429|
|  17|  13.6964785939759|3.7107776356656084|
|  18|13.011186262066634| 3.466850838939892|
|  19|12.8

In [7]:
tdf.groupBy("hour").avg("fare_amount", "h_distance").orderBy("hour").explain("formatted")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (7)
+- Sort (6)
   +- Exchange (5)
      +- HashAggregate (4)
         +- Exchange (3)
            +- HashAggregate (2)
               +- InMemoryTableScan (1)
                     +- InMemoryRelation (2)
                           +- * GpuColumnarToRow (4)
                              +- GpuBatchScan (3)


(1) InMemoryTableScan
Output [3]: [fare_amount#9, hour#16, h_distance#17]
Arguments: [fare_amount#9, hour#16, h_distance#17]

(2) InMemoryRelation
Arguments: [vendor_id#0, passenger_count#1, trip_distance#2, pickup_longitude#3, pickup_latitude#4, rate_code#5, store_and_fwd#6, dropoff_longitude#7, dropoff_latitude#8, fare_amount#9, trip_time#10L, year#11, month#12, day#13, day_of_week#14, is_weekend#15, hour#16, h_distance#17], CachedRDDBuilder(true,10000,StorageLevel(disk, memory, deserialized, 1 replicas),*(1) GpuColumnarToRow false
+- GpuBatchScan[vendor_id#0, passenger_count#1, trip_distance#2, pickup_longitude#3, pickup_latitude#4, rate_cod

In [8]:
val df3 = tdf.groupBy("hour").count

df3.orderBy(asc("hour"))show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df3: org.apache.spark.sql.DataFrame = [hour: int, count: bigint]
+----+------+
|hour| count|
+----+------+
|   0|319724|
|   1|225932|
|   2|169677|
|   3|120881|
|   4| 92592|
+----+------+
only showing top 5 rows



In [35]:
df3.explain("formatted")

In [17]:
tdf.groupBy("hour").count.orderBy("hour").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|hour| count|
+----+------+
|   0|319724|
|   1|225932|
|   2|169677|
|   3|120881|
|   4| 92592|
|   5| 90334|
|   6|201631|
|   7|336232|
|   8|403258|
|   9|397340|
|  10|384525|
|  11|396518|
|  12|422063|
|  13|424141|
|  14|444816|
|  15|437362|
|  16|398199|
|  17|474586|
|  18|562734|
|  19|556243|
+----+------+
only showing top 20 rows



In [19]:
%%sql
select * from taxi

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
%%sql
select trip_distance, rate_code, fare_amount, is_weekend, day_of_week from taxi


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sql
select trip_distance,avg(trip_distance),  avg(fare_amount)
from taxi
group by trip_distance order by avg(trip_distance) desc


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [15]:
%%sql
select hour, avg(fare_amount), avg(trip_distance)
from taxi
group by hour order by hour 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sql
select rate_code, avg(fare_amount) , avg(trip_distance)
from taxi
group by rate_code order by rate_code


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
select hour, avg(fare_amount), avg(h_distance)
from taxi
group by hour order by hour


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()